In [1]:
import os
import json
import openai

import data_utils

In [2]:
dataset = "chestxray"
prompt_type = "important"

In [3]:
openai.api_key = open(".openai_api_key", "r").read()[:-1]

In [4]:
prompts = {
    "important" : "List the most important features for recognizing something as \"Atelectasis\":\n\n-collapsed lung\n-reduced lung volume\n-displacement of interlobar fissures\n\nList the most important features for recognizing something as \"Consolidation\":\n\n-alveolar space filled with fluid\n-air bronchograms\n-homogeneous increase in pulmonary parenchymal attenuation\n\nList the most important features for recognizing something as \"{}\":",
    "superclass" : "Give superclasses for the word \"Atelectasis\":\n\n-lung disease\n-respiratory condition\n-medical condition\n\nGive superclasses for the word \"Consolidation\":\n\n-lung disease\n-respiratory condition\n-medical condition\n\nGive superclasses for the word \"{}\":",
    "around" : "List the things most commonly seen around \"Atelectasis\":\n\n-chest x-ray\n-lung\n-bronchi\n\nList the things most commonly seen around \"Consolidation\":\n\n-chest x-ray\n-lung\n-bronchi\n\nList the things most commonly seen around \"{}\":"
}

base_prompt = prompts[prompt_type]

In [5]:
cls_file = data_utils.LABEL_FILES[dataset]
with open(cls_file, "r") as f:
    classes = f.read().split("\n")

In [6]:
feature_dict = {}

for i, label in enumerate(classes):
    feature_dict[label] = set()
    print("\n", i, label)
    for _ in range(2):
        response = openai.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=[
                  {"role": "user", "content": base_prompt.format(label)}
              ],
              temperature=0.7,
              max_tokens=256,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )

        features = response.choices[0].message.content
        features = features.split("\n-")
        features = [feat.replace("\n", "") for feat in features]
        features = [feat.strip() for feat in features]
        features = [feat for feat in features if len(feat)>0]
        features = set(features)
        feature_dict[label].update(features)
    feature_dict[label] = sorted(list(feature_dict[label]))


 0 Atelectasis


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-*******************************************************************************************************************************************************4xWk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [7]:
json_object = json.dumps(feature_dict, indent=4)
with open("data/concept_sets/gpt3_init/gpt3_{}_{}_new.json".format(dataset, prompt_type), "w") as outfile:
    outfile.write(json_object)